In [ ]:
# Instalamos Selenium y el driver de Chrome para realizar pruebas del navegador
!pip install selenium
!apt get update 
!apt install chronium-chromedriver
!cp /usr/lib/chronium-browser/chromedriver /usr/bin

     |████████████████████████████████| 958 kB 23.6 MB/s 
     |████████████████████████████████| 356 kB 34.4 MB/s 
     |████████████████████████████████| 138 kB 49.4 MB/s 
     |████████████████████████████████| 3.6 MB 39.1 MB/s 
     |████████████████████████████████| 55 kB 4.2 MB/s 
     |████████████████████████████████| 54 kB 4.4 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.7 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
E: Invalid operation get
Reading package lists... Done
Building dependency tree       
Reading state information.

In [ ]:
# Instalamos Kora para importar funciones de un .ipynb a otro .ipynb en Google COLAB
!pip install kora -q

     |████████████████████████████████| 57 kB 6.2 MB/s 
     |████████████████████████████████| 56 kB 5.1 MB/s 


In [ ]:
# Instalamos Beautiful Soup
pip install BeautifulSoup4

In [ ]:
# Una vez instaladas las librerias necesarias, procedemos a su llamado para este proyecto Colab
from bs4 import BeautifulSoup
from selenium import webdriver
from kora.selenium import wd
# Importamos Pandas para el trabajo con los dataframe y Time para controlar tiempos de ejecucion
import sys
import pandas as pd
from datetime import date
import time

In [ ]:
# Usamos el driver de Chrome en Colab
sys.path.insert(0,'/usr/lib/chronium-browser/chromedriver')
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',options=options)
wd.get('https://simple.ripley.com.pe/')
wd.title

In [ ]:
# Declaramos la Pagina Web a scrapear
home_link = 'https://simple.ripley.com.pe/'
search_kw= 'laptops'.replace('','+')

In [ ]:
driver.get(home_link+'/tecnologia/computacion/laptops?source=search_kw&page=0')

In [ ]:
# Declaramos listas vacias para cada columna que de a cuerdo al detalle de los productos encontrados en el Pagina Web
laptops_marca=[]
laptops_descripcion=[]
laptops_precioNormal=[]
laptops_precioInternet=[]
laptops_precioRipley=[]
laptops_descuento=[]

In [ ]:
# Con una estructura For hacemos un recorrido de cada etiqueta con la respectiva clase que contiene el texto con la informacion para la columna que deseamos
pg_amount = 19
page = BeautifulSoup(driver.page_source, 'html.parser')

for i in range(0, pg_amount):
  for laptops in page.findAll('div', class_='catalog-product-item catalog-product-item__container col-xs-6 col-sm-6 col-md-4 col-lg-4'):
      marca = laptops.find('div', attrs={'class': 'catalog-product-details__logo-container'})
      if marca:
        laptops_marca.append(marca.text)
      else:  laptops_marca.append('')

      descripcion = laptops.find('div', attrs={'class': 'catalog-product-details__name'})
      if descripcion: 
        laptops_descripcion.append(descripcion.text)
      else: laptops_descripcion.append('')

      precioNormal=laptops.find('li', attrs={'class': 'catalog-prices__list-price catalog-prices__lowest catalog-prices__line_thru'})
      if precioNormal: 
        laptops_precioNormal.append(precioNormal.text)
      else: laptops_precioNormal.append('')

      precioInternet=laptops.find('li', attrs={'class': 'catalog-prices__offer-price'})
      if precioInternet: 
        laptops_precioInternet.append(precioInternet.text)
      else: laptops_precioInternet.append('')

      precioRipley=laptops.find('li', attrs={'class': 'catalog-prices__card-price'})
      if precioRipley: 
        laptops_precioRipley.append(precioRipley.text)
      else: laptops_precioRipley.append('')

      descuento=laptops.find('div', attrs={'class': 'catalog-product-details__discount-tag'})
      if descuento: 
        laptops_descuento.append(descuento.text)
      else: laptops_descuento.append('')

# asigamos un tiempo de ejecucion por cada interaccion del recorrido para evitar que la pagina detecte como bots
next_btn = driver.find_elements_by_css_selector('.pagination')
time.sleep(5)
driver.quit()
 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:37: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead


In [ ]:
# Le asignamos un nombre a cada columna 
laptops_list = pd.DataFrame({
                               'marca': laptops_marca,
                               'descripcion': laptops_descripcion,
                               'precioNormal': laptops_precioNormal,
                               'precioInternet': laptops_precioInternet,
                               'precioRipley': laptops_precioRipley,
                               'descuento': laptops_descuento
                              })


laptops_list.to_csv('artefactosRipley.csv', index=None, header=True, encoding='utf-8-sig' )

laptops_list

,marca,descripcion,precioNormal,precioInternet,precioRipley,descuento
0,TOSHIBA,TOSHIBA CANVIO BASICS DISCO EXTERNO,S/ 249,S/ 199,,-20%
1,HP,"LAPTOP HP 14-CF2074LA 8GB DDR4 256GB SSD 14""","S/ 2,599","S/ 2,299","S/ 2,299",-12%
2,ASUS,LAPTOP ASUS X515EA-BQ1001T INTEL CORE I5 8GB R...,"S/ 3,699","S/ 2,499","S/ 2,399",-35%
3,ACER,LAPTOP ACER A315-57G-53R9 INTEL CORE I5 1035G1...,"S/ 3,199","S/ 2,699","S/ 2,599",-19%
4,ASUS,LAPTOP ASUS X415 X415EA-EB112T 8GB DDR4 256GB ...,"S/ 2,599","S/ 2,199","S/ 2,099",-19%
...,...,...,...,...,...,...
907,ACER,LAPTOP ACER AN515-55-789Q INTEL CORE I7 10750H...,"S/ 5,799","S/ 5,699",,-2%
908,ASUS,LAPTOP ASUS ZENBOOK 14 UX425EA-KI355T INTEL CO...,"S/ 4,399","S/ 3,349",,-24%
909,ASUS,LAPTOP ASUS VIVOBOOK 15 OLED K513EA-L11073T IN...,"S/ 5,599","S/ 4,599",,-18%
910,HP,LAPTOP GAMER HP 15-DK1045LA INTEL CORE I7 DDR4...,"S/ 5,599","S/ 4,999",,-11%


In [ ]:
# leemos el dataframe en .csv para comprobar resultados
frm = pd.read_csv('artefactosRipley.csv')
frm

,marca,descripcion,precioNormal,precioInternet,precioRipley,descuento
0,HP,"LAPTOP HP 14-CF2074LA 8GB DDR4 256GB SSD 14""","S/ 2,599","S/ 2,399","S/ 2,199",-15%
1,ASUS,LAPTOP ASUS X515EA-BQ1001T INTEL CORE I5 8GB R...,"S/ 3,699","S/ 2,499","S/ 2,399",-35%
2,ACER,LAPTOP ACER A315-57G-53R9 INTEL CORE I5 1035G1...,"S/ 3,199","S/ 2,699","S/ 2,599",-19%
3,ASUS,LAPTOP ASUS X415 X415EA-EB112T 8GB DDR4 256GB ...,"S/ 2,599","S/ 2,199","S/ 2,099",-19%
4,APPLE,"LAPTOP APPLE MACBOOK AIR 8GB DDR4 256GB SSD 13""","S/ 5,799","S/ 4,599","S/ 4,499",-22%
...,...,...,...,...,...,...
907,ASUS,LAPTOP ASUS ZENBOOK 14 UX425EA-KI355T INTEL CO...,"S/ 4,399","S/ 3,349","S/ 3,149",-28%
908,ASUS,LAPTOP ASUS VIVOBOOK 15 OLED K513EA-L11073T IN...,"S/ 5,599","S/ 4,599","S/ 4,399",-21%
909,HP,LAPTOP GAMER HP 15-DK1045LA INTEL CORE I7 DDR4...,"S/ 5,599","S/ 4,999","S/ 4,799",-14%
910,LENOVO,LAPTOP LENOVO YOGA C940 INTEL CORE I7 12GB 256...,"S/ 6,999","S/ 5,199","S/ 4,999",-29%


In [ ]:
# Descargamos el Dataframe en formato .csv
from google.colab import files
files.download('artefactosRipley.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>